In [1]:
 from __future__ import print_function
import tensorflow as tf
import os
import import_ipynb
from skimage.transform import resize
from skimage.io import imsave
from data import load_test_data , load_train_data

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ELU, LeakyReLU,UpSampling2D, Input,add, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense , Dropout, BatchNormalization, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.activations import softmax
#from keras.layers.advanced_activations import ELU, LeakyReLU
 
 
 
 
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
 
img_rows = 96
img_cols = 96
 
smooth = 1.


def NConv2D(filters, kernel_size, strides=(1, 1), padding='valid', dilation_rate=1,
            activation=None, kernel_initializer='glorot_uniform'):
    
    
    actv = activation == 'relu' and (lambda: LeakyReLU(0.0)) or activation == 'elu' and (lambda: ELU(1.0)) or None

    def f(_input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                      dilation_rate=dilation_rate, kernel_initializer=kernel_initializer)(_input)
        norm = BatchNormalization(axis=3)(conv)
        return actv()(norm)

    return f

def inception_block_v2(inputs, filters, activation=None, version='b', pars={}, allowed_pars={}):
   
    actv = activation == 'relu' and (lambda: LeakyReLU(0.0)) or activation == 'elu' and (lambda: ELU(1.0)) or None

    c1_1 = Conv2D(filters=filters // 16, kernel_size=(1, 1), padding='same',
                  activation=activation, kernel_initializer='he_normal')(inputs)
    
    c1_2 = NConv2D(filters=filters // 8, kernel_size=(1, 3), padding='same',
                   activation=activation, kernel_initializer='he_normal')(c1_1)
    c1_3 = NConv2D(filters=filters // 8, kernel_size=(3, 1), padding='same',
                   activation=activation, kernel_initializer='he_normal')(c1_2)
    c1_4 = NConv2D(filters=filters // 8, kernel_size=(1, 3), padding='same',
                   activation=activation, kernel_initializer='he_normal')(c1_3)
    
    c1 = Conv2D(filters=filters // 8, kernel_size=(3, 1), padding='same', kernel_initializer='he_normal')(c1_4)
    

    # vertical 2
    c2_1 = Conv2D(filters=filters // 8 * 3, kernel_size=(1, 1), padding='same',
                  activation=activation, kernel_initializer='he_normal')(inputs)
        
    c2_2 = NConv2D(filters=filters // 2, kernel_size=(1, 3), padding='same',
                   activation=activation, kernel_initializer='he_normal')(c2_1)
                  
    c2 = Conv2D(filters=filters // 2, kernel_size=(3, 1), padding='same', kernel_initializer='he_normal')(c2_2)
    

    # vertical 3
    p3_1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    c3 = Conv2D(filters=filters // 8, kernel_size=(1, 1), padding='same', kernel_initializer='he_normal')(p3_1)

    # vertical 4
    c4 = Conv2D(filters=filters // 4, kernel_size=(1, 1), padding='same', kernel_initializer='he_normal')(inputs)

    # concatenating verticals together, normalizing and applying activation
    result = concatenate([c1, c2, c3, c4], axis=3)
    result = BatchNormalization(axis=3)(result)
    result = actv()(result)
    return result
                  
                  
def pooling_block(inputs, filters, kernel_size=(3, 3), strides=(2, 2), padding='same', activation=None,
                  pool_size=(2, 2), trainable=True, pars={}, allowed_pars={}):
    
    # checking that the allowed trainable parameters did not change in ALLOWED_PARS
   
    # keep trainable argument if need to use without PARS
    

    # setting the version from pars
    

    # returning block's output
    if trainable:
        return NConv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                       padding=padding, activation=activation)(inputs)
    else:
        return MaxPooling2D(pool_size=pool_size, padding=padding)(inputs)


def information_block(inputs, filters, kernel_size=(3, 3), padding='valid', activation=None,
                      block='inception', block_type='v2', version='b', pars={}, allowed_pars={}):
    
    # getting which block, block_type, version to use as the information block
    

    # inception block
 
    return inception_block_v2(inputs=inputs, filters=filters, activation=activation,
                                      version=version, pars=pars, allowed_pars=allowed_pars)
                  
def _shortcut(_input, residual):
    stride_width = _input.shape[1] / residual.shape[1]
    stride_height = _input.shape[2] / residual.shape[2]
    equal_channels = residual.shape[3] == _input.shape[3]

    shortcut = _input
    x = residual.shape[3]
    print('data type',type(stride_width),type(stride_height))
    print("type of data",x,type(x))
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=x, kernel_size=(1, 1),
                          strides=(int(stride_width), int(stride_height)),
                          kernel_initializer="he_normal", padding="valid")(_input)

    return add([shortcut, residual])
                  
                  

def rblock(inputs, filters, kernel_size, padding='valid', activation=None, scale=0.1):
    
    actv = activation == 'relu' and (lambda: LeakyReLU(0.0)) or activation == 'elu' and (lambda: ELU(1.0)) or None

    residual = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding)(inputs)
    residual = BatchNormalization(axis=3)(residual)
    residual = Lambda(lambda x: x * scale)(residual)
    res = _shortcut(inputs, residual)
    return actv()(res)
        
  

def connection_block(inputs, filters, padding='valid', activation=None,
                     version='residual', pars={}, allowed_pars={}):
  
    return rblock(inputs=inputs, filters=32, kernel_size=(1, 1), padding='same', activation=activation)
    
                  
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
 
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)                  


def get_unet(pars,allowed_pars):


    inputs = Input((img_rows, img_cols, 1))
    print('inputs:', inputs.shape)


    conv1 = information_block(inputs, 32, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv1', conv1.shape)
    pool1 = pooling_block(inputs=conv1, filters=32, activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('pool1', pool1.shape)
    pool1 = Dropout(0.5)(pool1)
    print('pool1', pool1.shape)

    conv2 = information_block(pool1, 64, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv2', conv2.shape)
    pool2 = pooling_block(inputs=conv2, filters=64, activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('pool2', pool2.shape)
    pool2 = Dropout(0.5)(pool2)
    print('pool2', pool2.shape)

    conv3 = information_block(pool2, 128, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv3', conv3.shape)
    pool3 = pooling_block(inputs=conv3, filters=128, activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('pool3', pool3.shape)
    pool3 = Dropout(0.5)(pool3)
    print('pool3', pool3.shape)

    conv4 = information_block(pool3, 256, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv4', conv4.shape)
    pool4 = pooling_block(inputs=conv4, filters=256, activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('pool4', pool4.shape)
    pool4 = Dropout(0.5)(pool4)
    print('pool4', pool4.shape)

    #
    # bottom level of the U-net
    #
    conv5 = information_block(pool4, 512, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv5', conv5.shape)
    conv5 = Dropout(0.5)(conv5)
    print('conv5', conv5.shape)

                  
                  
    
    print(tf.shape(conv5))
    td=conv5
    
    tx=softmax(td,axis=[1,2])
    q=1.31
    
    
    print("value of q",q)
    
    
    
    safe_x = K.maximum(tx,1e-6)
    new_p = K.switch(K.equal(q,1.), (K.log(safe_x)*safe_x)/K.log(2.718),(K.pow(safe_x,q))/(q-1))
    new_p = -new_p #-p^q/(q-1)
    
    if q!=1:
        new_p= 1+new_p
        
    print("new_p",new_p.shape)
    xyz=K.max(new_p,axis=[1,2],keepdims=True)
    print("xyz=",xyz.shape)
   
    
    print("xyz=",xyz.shape)
    weights=1-(new_p/xyz)
    print("weights",weights.shape)
    improved_p=td*weights
    print("improved_p=",improved_p.shape)
    improved_p=K.sum(improved_p,axis=[1,2])/144
    print("improved_pv2=",type(improved_p),improved_p.shape)


    #dense1=Dense(32,activation='relu')(improved_p)
    #print("dense1",dense1)
    dense2=Dense(1,activation='softmax')(improved_p)
    print("below dense",dense2)
                  
                  
                  
                  
                  

    after_conv4 = connection_block(conv4, 256, padding='same', activation='elu',
                                   pars=pars, allowed_pars=allowed_pars)
    print('after_conv4', after_conv4.shape)
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), after_conv4], axis=3)
    conv6 = information_block(up6, 256, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv6', conv6.shape)
    conv6 = Dropout(0.5)(conv6)
    print('conv6', conv6.shape)

    after_conv3 = connection_block(conv3, 128, padding='same', activation='elu',
                                   pars=pars, allowed_pars=allowed_pars)
    print('after_conv3', after_conv3.shape)
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), after_conv3], axis=3)
    conv7 = information_block(up7, 128, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv7', conv7.shape)
    conv7 = Dropout(0.5)(conv7)
    print('conv7', conv7.shape)

    after_conv2 = connection_block(conv2, 64, padding='same', activation='elu', pars=pars,
                                   allowed_pars=allowed_pars)
    print('after_conv2', after_conv2.shape)
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), after_conv2], axis=3)
    conv8 = information_block(up8, 64, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv8', conv8.shape)
    conv8 = Dropout(0.5)(conv8)
    print('conv8', conv8.shape)

    after_conv1 = connection_block(conv1, 32, padding='same', activation='elu',
                                   pars=pars, allowed_pars=allowed_pars)
    print('after_conv1', after_conv1.shape)
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), after_conv1], axis=3)
    conv9 = information_block(up9, 32, padding='same', activation='elu', pars=pars, allowed_pars=allowed_pars)
    print('conv9', conv9.shape)
    conv9 = Dropout(0.5)(conv9)
    print('conv9', conv9.shape)

    # main output
    conv10 = Conv2D(1, kernel_size=(1, 1), kernel_initializer='he_normal', activation='sigmoid', name='main_output')(
        conv9)
    print('conv10', conv10.shape)

    # creating a model
    # compiling the model

    model = Model(inputs=inputs, outputs=[conv10, dense2]) #edit here for Tsallis
    model.compile(optimizer=Adam(learning_rate=1e-5),
                  loss=[dice_coef_loss, tf.keras.losses.BinaryCrossentropy()],
                      metrics=[dice_coef,tf.keras.metrics.BinaryCrossentropy()],
                      loss_weights=[1., 0.5])

    return model
                  



def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
 
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

def train_and_predict():
                  
    BEST_PARS = {
    'outputs': 2,
    'activation': 'elu',
    'pooling_block': {'trainable': True},
    'information_block': {'inception': {'v2': 'b'}},
    'connection_block': 'residual'}
    
    ALLOWED_PARS = {
    'outputs': [1, 2],
    'activation': ['elu', 'relu'],
    'pooling_block': {
        'trainable': [True, False]},
    'information_block': {
        'inception': {
            'v1': ['a', 'b'],
            'v2': ['a', 'b', 'c'],
            'et': ['a', 'b']},
        'convolution': {
            'simple': ['not_normalized', 'normalized'],
            'dilated': ['not_normalized', 'normalized']}},
    'connection_block': ['not_residual', 'residual']}
                
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()
 
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
 
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
 
    imgs_train -= mean
    imgs_train /= std
 
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
 
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet(BEST_PARS,ALLOWED_PARS)
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss' , mode = 'min' , verbose = 1 , save_best_only=True )
    early_stopping = EarlyStopping(patience=5, verbose=1)
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train,imgs_value_train], batch_size=64, epochs= 40, verbose=1, shuffle=True,
              validation_split=0.2 ,  callbacks=[model_checkpoint,early_stopping])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    n_imgs_test = imgs_test.shape[0]
    #for i in range(n_imgs_test):

    #worked for 7 vs 1
    #imgs_mask_test, imgs_value_test= model.predict(np.expand_dims(imgs_test[0] , axis = 0), verbose=1)


    imgs_mask_test, imgs_value_test= model.predict(imgs_test, verbose=1)
    print("mask final img 1" , imgs_mask_test)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    
    # print(imgs_mask_test.shape , imgs_value_test.shape)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)


 
     
if __name__ == '__main__':
    train_and_predict()
        


importing Jupyter notebook from data.ipynb
------------------------------
Creating and compiling model...
------------------------------
inputs: (None, 96, 96, 1)
conv1 (None, 96, 96, 32)
pool1 (None, 48, 48, 32)
pool1 (None, 48, 48, 32)
conv2 (None, 48, 48, 64)
pool2 (None, 24, 24, 64)
pool2 (None, 24, 24, 64)
conv3 (None, 24, 24, 128)
pool3 (None, 12, 12, 128)
pool3 (None, 12, 12, 128)
conv4 (None, 12, 12, 256)
pool4 (None, 6, 6, 256)
pool4 (None, 6, 6, 256)
conv5 (None, 6, 6, 512)
conv5 (None, 6, 6, 512)
KerasTensor(type_spec=TensorSpec(shape=(4,), dtype=tf.int32, name=None), inferred_value=[None, 6, 6, 512], name='tf.compat.v1.shape/Shape:0', description="created by layer 'tf.compat.v1.shape'")
value of q 1.31
new_p (None, 6, 6, 512)
xyz= (None, 1, 1, 512)
xyz= (None, 1, 1, 512)
weights (None, 6, 6, 512)
improved_p= (None, 6, 6, 512)
improved_pv2= <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'> (None, 512)
below dense KerasTensor(type_spec=TensorSpec(shape=(None, 

ResourceExhaustedError:  OOM when allocating tensor with shape[64,96,96,96] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/concatenate_11/concat (defined at <ipython-input-1-695a28f923a4>:362) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_16891]

Function call stack:
train_function
